In [9]:
import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt
import seaborn as sns
import json
import psycopg2

In [10]:
with open("config.json") as f:
    connection=psycopg2.connect(json.load(f)["psql"])

In [11]:
%%time
query="""
SELECT
    ip,
    app,
    device,
    os,
    channel,
    is_attributed,
    EXTRACT(DOW FROM click_time) AS dow,
    EXTRACT(DOY FROM click_time) AS doy,
    EXTRACT(DAY FROM click_time) AS day
FROM click_data WHERE click_id IS NULL;
"""
train = pd.read_sql_query(query, connection)

CPU times: user 9min 20s, sys: 1min 48s, total: 11min 9s
Wall time: 16min 27s


In [7]:
%%time
query="""
SELECT
    ip,
    app,
    device,
    os,
    channel,
    click_id,
    EXTRACT(DOW FROM click_time) AS dow,
    EXTRACT(DOY FROM click_time) AS doy,
    EXTRACT(DAY FROM click_time) AS day
FROM click_data WHERE click_id IS NOT NULL
"""
test = pd.read_sql_query(query, connection)

CPU times: user 57.8 s, sys: 8.56 s, total: 1min 6s
Wall time: 4min 1s


In [13]:
gc.collect()

0